In [9]:
import torch
import numpy as np
from PIL import Image, ImageDraw
from torchvision.transforms.functional import to_pil_image
from facenet_pytorch.models.mtcnn import PNet, RNet, ONet  # Import P-Net, R-Net, O-Net
from facenet_pytorch import MTCNN
import os
#import matplotlib.pyplot as plt

# 1. Define fine-tuned P-Net, R-Net, and O-Net for finetuning
class FinetunedMTCNN(MTCNN):
#(self, image_size=160, margin=5, **kwargs): # use this after some time to improve the final results
    def __init__(self, image_size=160, margin=15, **kwargs):
        super(FinetunedMTCNN, self).__init__(**kwargs)

        # Create your custom, finetuned P-Net, R-Net, O-Net here
        self.pnet = PNet()
        self.rnet = RNet()
        self.onet = ONet()


    def forward(self, x):
        # Overriding forward pass if additional finetuning is needed
        return super().forward(x)



#new
finetuned_mtcnn = FinetunedMTCNN(keep_all=True, device='cuda:0' if torch.cuda.is_available() else 'cpu', min_face_size=20)
#new
# 2. Function to perform face detection, apply NMS, and save cropped face images
def process_and_save_faces(image_folder, mtcnn_model):
    global new_hey
    new_hey = []
    for filename in os.listdir(image_folder):
        if filename.endswith(".jpeg"):
            image_path = os.path.join(image_folder, filename)
            image = Image.open(image_path)

            # Detect faces (with bounding boxes, probabilities, and landmarks)
            boxes, probs, landmarks = mtcnn_model.detect(image, landmarks=True)
            #print(f"box{boxes} prob{probs} landmarks{landmarks}")
            #print(boxes)
            #print(probs)
            #print(landmarks)
            if boxes is not None:
                # Set a confidence threshold
                threshold = 0.95

                # Filter detected faces based on the confidence score
                filtered_faces = [i for i, prob in enumerate(probs) if prob > threshold]

                # Save cropped faces that pass the confidence threshold
                for i in filtered_faces:
                    box = boxes[i]  # Get the bounding box for the filtered face
                    # Crop the face using the O-Net bounding box
                    cropped_face = image.crop(box)
                    # Save the cropped face image in the output folder
                    #face_output_path = os.path.join(output_folder, f'face_{filename}_{i}.jpeg')
                    img = np.array(cropped_face)
                    new_hey.append(img)
    print(f"number of images when min_face_size = {len(new_hey)}")                

                    # Optionally visualize the stages
                    #visualize_detection_stages(image, boxes, landmarks, f'Stage {i+1}')

match_images_folder = r'D:\Images\Facial_reco_images\Facial_reco_images\Game'  # match photos
studio_images_folder = r'D:\Images\Facial_reco_images\Facial_reco_images\Players\Studio_photos'  # studio photos
# match_images_folder = '/content/drive/My Drive/match_photos_251'  #match photos
# output_folder_match = '/content/drive/My Drive/output_match'  #saving match photos
# studio_images_folder = '/content/drive/My Drive/studio_photos'   #studio photos
# output_folder_studio = '/content/drive/My Drive/output_studio'  #saving studio photos

# Create the output directories if they don't exist
# os.makedirs(output_folder_match, exist_ok=True)  # Create output_folder_match
# os.makedirs(output_folder_studio, exist_ok=True)

# 4. Process both match and studio images
process_and_save_faces(match_images_folder, finetuned_mtcnn)
#process_and_save_faces(studio_images_folder, finetuned_mtcnn)

number of images when min_face_size = 294
